In [2]:
%%capture

#installing athena and redshift connectors
import sys
!{sys.executable} -m pip install PyAthena;
!{sys.executable} -m pip install redshift_connector

# importing remaining packages
from pyathena import connect
import pandas as pd
from scipy import stats
pd.options.mode.chained_assignment = None 
import redshift_connector
import numpy as np
import os  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Connecting to athena

In [3]:
# connecting to athena
conn = connect(s3_staging_dir='s3://gd-revrelvnc-stage-athena-results/',
               region_name='us-west-2')

# opening SQL file
fd = open('athena_v2_nc.sql', 'r')
sqlFile = fd.read()
fd.close()

# running athena sql and storing in dataframe
athena_df = pd.read_sql(sqlFile,conn)
athena_df.head()

/tmp/ipykernel_106/2925443478.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  athena_df = pd.read_sql(sqlFile,conn)


,order_id,v2_18m
0,1836617333,34.066667
1,1838254165,45.513333
2,1837051783,839.000000
3,1838305980,60.040000
4,1837619212,2.376667


In [4]:
query = """
select 
    distinct
    merged_shopper_id as shopper_id
from analytic.ads_bill_line_ext
where partition_bill_mst_year_month >= substring(cast(DATE_ADD('day', 0, cast('2022-02-01' as date)) as varchar), 1, 7)
    and bill_modified_mst_date between DATE_ADD('day', 1, cast('2022-02-01' as date)) and DATE_ADD('month', 2, cast('2022-02-01' as date))
    and bill_fraud_flag = FALSE
    and chargeback_flag = FALSE
    and COALESCE(original_shopper_exclude_reason_desc, exclude_reason_desc) IS NULL
    and new_acquisition_flag = True
    limit 10
"""
df = pd.read_sql(query, conn)
df

/tmp/ipykernel_106/254526393.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,shopper_id
0,438009979
1,461244253
2,462807640
3,464052902
4,447412333
5,456643209
6,464817313
7,462591213
8,454190683
9,464727404


In [12]:
from pyathena import connect
import time
import pandas as pd

def read_sql(query):
    start_time = time.time()
    connection = connect(
                   s3_staging_dir='s3://gd-revrelvnc-stage-athena-results/',
                   region_name='us-west-2')
    df = pd.read_sql(query, connection)
    print('running {} second to read the data with data size {} and {} columns'.format(int(time.time() - start_time), df.shape[0], df.shape[1]))
    return df

In [13]:
query = """
select bill_id,bill_line_num from analytic.ads_bill_line_ext limit 3
"""
df = read_sql(query)
df.head()

/tmp/ipykernel_106/4056930099.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


running 4 second to read the data with data size 3 and 2 columns


,bill_id,bill_line_num
0,141659,0
1,141914-R,1
2,141076,1


## 2. Connecting to redshift

In [5]:
# connecting to redshift
conn = redshift_connector.connect(
    host='ckp-analytics.ci1mx4gxg5cn.us-west-2.redshift.amazonaws.com',
    database='bi',
    user='user_id',
    password='password')
cursor = conn.cursor()

# opening SQL file
fd = open('redshift_nc.sql', 'r')
sqlFile = fd.read()
fd.close()

# executing redshift sql
cursor.execute(sqlFile)
redshift_df = cursor.fetch_dataframe()
redshift_df.head()

,order_id,parent_channel_name,campaign_name,actual_18m,v1_18m
0,1836134339,Paid Search - Brand,pt-br_corp-core_sem_bh_b_x_aware-consider_x_pr...,102.77,50.586702
1,1836890576,Paid Search - Brand,en-in_corp-core_sem_bh_b_x_aware-consider_m_pr...,1.93,3.900417
2,1838106805,Display,en-us_multi_display_x_ret_gd_us_conv-DV360_003,39.98,68.440871
3,1837650972,Paid Search - Brand,vi-vn_corp-core_sem_bh_b_x_aware-consider_x_pr...,25.90,82.090065
4,1838551728,Display,en-uk_corp_display_x_pros_intl_x_001,268.74,138.371526


In [6]:
!{sys.executable} -m pip install psycopg2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import psycopg2
import pandas as pd
conn = psycopg2.connect(dbname='bi',
                        host='ckp-analytics.ci1mx4gxg5cn.us-west-2.redshift.amazonaws.com',
                        port='5439',
                        user='user_id',
                        password='password')


In [10]:
query = """
select 
    distinct
    merged_shopper_id as shopper_id,
    bill_id,
    bill_modified_mst_date as bill_date,
    new_acquisition_flag,
    new_registered_user_flag
from bi.analytic.ads_bill_line
where partition_bill_mst_year_month >= '2023-01'
    and bill_modified_mst_date between DATE_ADD('day', 0, cast('2023-01-01' as date)) and DATE_ADD('month', 2, cast('2023-01-01' as date))
    and pf_id in (850, 757)
    limit 20
"""
df = pd.read_sql(query, conn)
df

/opt/conda/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,shopper_id,bill_id,bill_date,new_acquisition_flag,new_registered_user_flag
0,106313985,2427695295,2023-01-04,False,False
1,456265187,2430755933,2023-01-20,True,False
2,558474021,2454692407,2023-01-29,True,False
3,455041460,2427677485,2023-01-04,False,False
4,8091830,2426938936,2023-01-03,False,False
5,555214749,2424211096,2023-01-22,False,False
6,24513785,2441050883R,2023-01-17,False,False
7,308547879,2430145151,2023-01-11,False,False
8,204228333,2440681005,2023-01-16,False,False
9,555248639,2431430683,2023-01-07,True,False
